## Importing Libraries

In [2]:
import tensorflow as tf
import os
import numpy as np

from matplotlib import pyplot as plt
%matplotlib inline

print("Tensorflow version:", tf.version.VERSION)

Tensorflow version: 2.12.0


## Preprocessing Data

In [6]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

print(X_train.shape, y_train.shape)

(60000, 28, 28) (60000,)


In [8]:
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

print(y_train.shape)
print(y_test.shape)

(60000, 10)
(10000, 10)


## Creating Model

In [25]:
def convBlock(num_filters):
    return tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(num_filters,
                            kernel_size=3,
                            padding='same',
                            activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(num_filters,
                            kernel_size=3, 
                            activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=2),
        tf.keras.layers.Dropout(rate=0.2),
    ])

model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(28, 28, 1)),
    convBlock(num_filters=32),
    convBlock(num_filters=64),
    convBlock(num_filters=128),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [26]:
model.summary(expand_nested=True)

Model: "sequential_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_23 (Sequential)  (None, 13, 13, 16)        2544      
|¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯|
| conv2d_34 (Conv2D)        (None, 28, 28, 16)        160       |
|                                                               |
| batch_normalization_17 (Bat  (None, 28, 28, 16)     64        |
| chNormalization)                                              |
|                                                               |
| conv2d_35 (Conv2D)        (None, 26, 26, 16)        2320      |
|                                                               |
| max_pooling2d_17 (MaxPoolin  (None, 13, 13, 16)     0         |
| g2D)                                                          |
|                                                               |
| dropout_17 (Dropout)      (None, 13, 13, 16)       

In [ ]:
X_train = X_train / 255.
X_test = X_test / 255.


history = model.fit(X_train,
                    y_train,
                    validation_data=(X_test, y_test),
                    epochs=10,
                    batch_size=128,
                    callbacks=[
                        tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                         patience=3),
                        tf.keras.callbacks.ModelCheckpoint('model_{val_accuracy:.3f}.h5',
                                                           save_best_only=True,
                                                           save_weights_only=False,
                                                           monitor='val_accuracy')
                    ])

In [ ]:
import pandas as pd

pd.DataFrame(history.history).plot(figsize=(6, 4))